In [60]:
import pandas as pd
import numpy as np
from fycharts.SpotifyCharts import SpotifyCharts
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import musicbrainzngs

In [29]:
'''api = SpotifyCharts()
api.top200Weekly(output_file = "top_200_weekly-missing.csv", start = "2020-12-11", end = "2020-12-31", region = ["pt"])'''

'api = SpotifyCharts()\napi.top200Weekly(output_file = "top_200_weekly-missing.csv", start = "2020-12-11", end = "2020-12-31", region = ["pt"])'

In [71]:
charts = pd.read_csv("top_200_weekly.csv")
charts = charts.dropna()

missing_charts = pd.read_csv("top_200_weekly-missing.csv")

charts = pd.concat([charts,missing_charts])
charts

,Position,Track Name,Artist,Streams,date,region,spotify_id
0,1.0,Starboy,The Weeknd,167258.0,2016-12-30--2017-01-06,pt,5aAx2yezTd8zXrkmtKl66Z
1,2.0,Say You Won't Let Go,James Arthur,113379.0,2016-12-30--2017-01-06,pt,5uCax9HTNlzGybIStD3vDh
2,3.0,Closer,The Chainsmokers,113157.0,2016-12-30--2017-01-06,pt,7BKLCZ1jbUBVqRi2FVlTVw
3,4.0,I Feel It Coming,The Weeknd,111042.0,2016-12-30--2017-01-06,pt,5GXAXm5YOmYT0kL5jHvYBt
4,5.0,Let Me Love You,DJ Snake,109443.0,2016-12-30--2017-01-06,pt,4pdPtRcBmOSQDlJ3Fk945m
...,...,...,...,...,...,...,...
395,196.0,Rainha da Favela,Ludmilla,39097.0,2020-12-18--2020-12-25,pt,3vu864xi5Xis9VfOsJpmXu
396,197.0,Sólido,Instinto 26,38983.0,2020-12-18--2020-12-25,pt,166Lpx4pr5vtIJYInQx4AJ
397,198.0,Tranquilo,Aragão,38804.0,2020-12-18--2020-12-25,pt,1fobXPCu47gikLMIoyQBLy
398,199.0,Reminds Me Of You,Juice WRLD,38487.0,2020-12-18--2020-12-25,pt,5YSHygGN9D0mID1NsVd5my


In [72]:
#fixing dates
charts["date"] = charts["date"].str.split("--")
charts["start_date"], charts["end_date"] = zip(*charts.pop('date'))

charts["start_date"] = pd.to_datetime(charts["start_date"], errors='coerce')
charts["end_date"] = pd.to_datetime(charts["end_date"], errors='coerce')

charts = charts.rename(columns={'Track Name': 'title', 'Artist': 'artist'})

In [76]:
charts.dtypes

Position             float64
title                 object
artist                object
Streams              float64
region                object
spotify_id            object
start_date    datetime64[ns]
end_date      datetime64[ns]
dtype: object

In [59]:
streams_by_artist = pd.pivot_table(charts, values="Streams",index=['Artist'], aggfunc=np.sum)
streams_by_artist = streams_by_artist.reset_index()
streams_by_artist.sort_values(by=["Streams"],ascending=False)

#streams_by_artist.to_csv("top_streams.csv")

In [34]:
# lista de artistas
unique_artists = list(charts["Artist"].unique())
charts["Artist"].value_counts()

Wet Bed Gang       980
Post Malone        958
Ed Sheeran         806
Piruka             784
XXXTENTACION       709
                  ... 
Michael Jackson      1
Céline Dion          1
Cesar Sampson        1
FTampa               1
KREAM                1
Name: Artist, Length: 751, dtype: int64

In [61]:
load_dotenv()

musicbrainzngs.auth(os.environ.get("MUSICBRAINZ_USER"), os.environ.get("MUSICBRAINZ_USER"))
musicbrainzngs.set_useragent("chart", 0.1, "https://rute.radio")

In [62]:
new_df = pd.DataFrame()

for artist in tqdm(unique_artists):
    artist_list = musicbrainzngs.search_artists(artist=artist)
    artist_df = pd.DataFrame.from_records(artist_list)
    new_df = pd.concat([new_df, artist_df])
    
print("Já está")


Já está


In [63]:
#retirar informação dos dicionários extraídos do musicbrainz


def get_name(x):
    if type(x) == dict:
        return x.get("name")
    
    return "---"

def get_country(x):
    if type(x) == dict:
        return x.get("country")
    
    return "---"
            
new_df["artist"] = new_df["artist-list"].apply(get_name)  
new_df["country"] = new_df["artist-list"].apply(get_country)
new_df = new_df.drop(['artist-list', "artist-count"], axis=1)
new_df = new_df.reset_index()
new_df = new_df.drop("index", axis=1)

In [73]:
newish = charts.merge(new_df, how="left", on='artist')
newish = newish.drop_duplicates()
newish = newish.reset_index()

newish.isna().sum()

index             0
Position          0
title             0
artist            0
Streams           0
region            0
spotify_id        0
start_date        0
end_date          0
country       11356
dtype: int64

In [78]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation["artist"].value_counts().head(30)

Drake               684
ProfJam             509
MC Kevinho          479
Khalid              460
Dillaz              400
Anitta              396
Deejay Telio        371
James Arthur        278
Bispo               271
Richie Campbell     259
Sam Smith           256
mishlawi            220
Papillon            196
Queen               190
DJ Snake            185
D.A.M.A             140
Future              135
LON3R JOHNY         134
Virgul              130
Bárbara Tinoco      129
Ludmilla            125
Phoenix Rdc         125
Giulia Be           122
Logic               111
Oasis               109
MC Kevin o Chris    106
DENNIS              106
Sia                 104
Supa Squad           88
Dj Dadda             84
Name: artist, dtype: int64

In [80]:
newish.loc[newish['artist'] == 'Drake', 'country'] = 'US'
newish.loc[newish['artist'] == 'ProfJam', 'country'] = 'PT'
newish.loc[newish['artist'] == 'MC Kevinho', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Khalid', 'country'] = 'US'
newish.loc[newish['artist'] == 'Dillaz', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Deejay Telio', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Anitta', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Bispo', 'country'] = 'PT'
newish.loc[newish['artist'] == 'James Arthur', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Sam Smith', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Richie Campbell', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Papillon', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Queen', 'country'] = 'GB'
newish.loc[newish['artist'] == 'DJ Snake', 'country'] = 'FR'
newish.loc[newish['artist'] == 'D.A.M.A', 'country'] = 'PT'
newish.loc[newish['artist'] == 'LON3R JOHNY', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Future', 'country'] = 'US'
newish.loc[newish['artist'] == 'Virgul', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Bárbara Tinoco', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Phoenix Rdc', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Giulia Be', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Logic', 'country'] = 'US'
newish.loc[newish['artist'] == 'Oasis', 'country'] = 'GB'
newish.loc[newish['artist'] == 'DENNIS', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Sia', 'country'] = 'AU'
newish.loc[newish['artist'] == 'Supa Squad', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Dj Dadda', 'country'] = 'PT'

In [81]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation["artist"].value_counts().head(30)

mishlawi              220
Ludmilla              125
MC Kevin o Chris      106
Mabel                  83
9 Miller               81
Twenty One Pilots      78
Apollo G               74
KAROL G                73
Regard                 73
Don Toliver            73
Tones And I            72
Chris Brown            72
Alan Walker            70
Miley Cyrus            69
Mc Livinho             68
G-Eazy                 67
Nio Garcia             66
Rihanna                64
MC JottaPê             64
Axwell /\ Ingrosso     63
Pk                     62
ANAVITÓRIA             62
Aragão                 62
Instinto 26            60
John Mayer             60
Rag'n'Bone Man         57
Kanye West             57
Chico da Tina          55
Sofia Reyes            54
MC Bruninho            51
Name: artist, dtype: int64

In [85]:
newish.loc[newish['artist'] == 'Twenty One Pilots', 'country'] = 'US'
newish.loc[newish['artist'] == 'mishlawi', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Ludmilla', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Mabel', 'country'] = 'GB'
newish.loc[newish['artist'] == '9 Miller', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Apollo G', 'country'] = 'PT'
newish.loc[newish['artist'] == 'MC Kevin o Chris', 'country'] = 'BR'
newish.loc[newish['artist'] == 'KAROL G', 'country'] = 'CO'
newish.loc[newish['artist'] == 'Regard', 'country'] = 'KO'
newish.loc[newish['artist'] == 'Don Toliver', 'country'] = 'US'
newish.loc[newish['artist'] == 'Tones And I', 'country'] = 'AU'
newish.loc[newish['artist'] == 'Chris Brown', 'country'] = 'US'
newish.loc[newish['artist'] == 'Alan Walker', 'country'] = 'GB/NO'
newish.loc[newish['artist'] == 'Miley Cyrus', 'country'] = 'US'
newish.loc[newish['artist'] == 'Mc Livinho', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Nio Garcia', 'country'] = 'US'
newish.loc[newish['artist'] == 'G-Eazy', 'country'] = 'US'
newish.loc[newish['artist'] == 'Rihanna', 'country'] = 'BA'
newish.loc[newish['artist'] == 'MC JottaPê', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Axwell /\ Ingrosso', 'country'] = 'SE'
newish.loc[newish['artist'] == 'Pk', 'country'] = 'BR'
newish.loc[newish['artist'] == 'ANAVITÓRIA', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Aragão', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Instinto 26', 'country'] = 'PT'
newish.loc[newish['artist'] == 'John Mayer', 'country'] = 'US'
newish.loc[newish['artist'] == 'Kanye West', 'country'] = 'US'
newish.loc[newish['artist'] == 'Chico da Tina', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Sofia Reyes', 'country'] = 'MX'
newish.loc[newish['artist'] == 'MC Bruninho', 'country'] = 'BR'

In [86]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation["artist"].value_counts().head(30)

Rag'n'Bone Man         57
Jimmy P                50
Mc Don Juan            50
Panic! At The Disco    49
Valete                 49
Machine Gun Kelly      47
Lil Wayne              46
Y2K                    46
Dynho Alves            45
Pineapple StormTv      45
Ricky Martin           44
Mustard                43
FRANKIEONTHEGUITAR     43
Tropkillaz             43
Jvst Fly               42
MC Fioti               41
Sam The Kid            40
PEDRO SAMPAIO          40
Vytinho NG             39
Kappa Jotta            36
Dadá Boladão           36
Surfaces               36
Offset                 35
StaySolidRocky         34
Edgar Domingos         34
Domingues              34
MC Ingryd              33
Tom Walker             33
Mc Th                  32
Pitbull                31
Name: artist, dtype: int64

In [89]:
newish.loc[newish['artist'] == 'Rag\'n\'Bone Man', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Jimmy P', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Mc Don Juan', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Panic! At The Disco', 'country'] = 'US'
newish.loc[newish['artist'] == 'Valete', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Machine Gun Kelly', 'country'] = 'US'
newish.loc[newish['artist'] == 'Lil Wayne', 'country'] = 'US'
newish.loc[newish['artist'] == 'Y2K', 'country'] = 'US'
newish.loc[newish['artist'] == 'Dynho Alves', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Pineapple StormTv', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Ricky Martin', 'country'] = 'US'
newish.loc[newish['artist'] == 'Mustard', 'country'] = 'US'

newish.loc[newish['artist'] == 'FRANKIEONTHEGUITAR', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Jvst Fly', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Tropkillaz', 'country'] = 'BR'

newish.loc[newish['artist'] == 'MC Fioti', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Sam The Kid', 'country'] = 'PT'
newish.loc[newish['artist'] == 'PEDRO SAMPAIO', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Dadá Boladão', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Vytinho NG', 'country'] = 'BR'

newish.loc[newish['artist'] == 'Surfaces', 'country'] = 'US'
newish.loc[newish['artist'] == 'Offset', 'country'] = 'US'
newish.loc[newish['artist'] == 'StaySolidRocky', 'country'] = 'US'
newish.loc[newish['artist'] == 'Edgar Domingos', 'country'] = 'PT/AN'
newish.loc[newish['artist'] == 'Domingues', 'country'] = 'PT'

newish.loc[newish['artist'] == 'MC Ingryd', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Tom Walker', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Mc Th', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Pitbull', 'country'] = 'US'

In [92]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation["artist"].value_counts().head(20)

Kappa Jotta                 36
Gson                        31
KALEO                       31
Adele                       30
A Boogie Wit da Hoodie      29
Sebastian Yatra             29
Lexa                        28
Rony Fuego                  28
Famous Dex                  27
Felipe Original             27
Silva                       26
SYRO                        26
Young T & Bugsey            26
Mc Jacaré                  25
Mizzy Miles                 24
Domi                        23
WUANT                       23
Russ                        23
Tyler, The Creator          22
MC's Jhowzinho & Kadinho    21
Name: artist, dtype: int64

In [93]:
newish.loc[newish['artist'] == 'Kappa Jotta', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Gson', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Adele', 'country'] = 'GB'
newish.loc[newish['artist'] == 'A Boogie Wit da Hoodie', 'country'] = 'US'
newish.loc[newish['artist'] == 'Sebastian Yatra', 'country'] = 'CO'
newish.loc[newish['artist'] == 'Lexa', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Rony Fuego', 'country'] = 'AN'
newish.loc[newish['artist'] == 'Famous Dex', 'country'] = 'US'
newish.loc[newish['artist'] == 'Felipe Original', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Silva', 'country'] = 'BR'
newish.loc[newish['artist'] == 'SYRO', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Young T & Bugsey', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Mc Jacaré', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Mizzy Miles', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Domi', 'country'] = 'PT'
newish.loc[newish['artist'] == 'WUANT', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Russ', 'country'] = 'US'
newish.loc[newish['artist'] == 'Tyler, The Creator', 'country'] = 'US'
newish.loc[newish['artist'] == 'MC\'s Jhowzinho & Kadinho ', 'country'] = 'BR'

In [96]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation["artist"].value_counts().head(20)

KALEO                       31
Mike Perry                  21
Joel Corry                  21
MC's Jhowzinho & Kadinho    21
Internet Money              20
Joint One                   20
Thiaguinho MT               20
Soraia Ramos                19
AJR                         19
Mastiksoul                  18
Rich The Kid                18
Toy                         17
FRENSHIP                    16
Silk City                   16
Vado Más Ki Ás              16
ANDREZO                     16
The Vamps                   15
LP                          15
DripReport                  15
Miguel                      14
Name: artist, dtype: int64

In [97]:
newish.loc[newish['artist'] == 'KALEO', 'country'] = 'IC'
newish.loc[newish['artist'] == 'Mike Perry', 'country'] = 'SE'
newish.loc[newish['artist'] == 'Joel Corry', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Internet Money', 'country'] = 'US'
newish.loc[newish['artist'] == 'Joint One', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Thiaguinho MT', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Soraia Ramos', 'country'] = 'AN'
newish.loc[newish['artist'] == 'AJR', 'country'] = 'US'
newish.loc[newish['artist'] == 'Mastiksoul', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Rich The Kid', 'country'] = 'US'
newish.loc[newish['artist'] == 'Toy', 'country'] = 'PT'

newish.loc[newish['artist'] == 'FRENSHIP', 'country'] = 'US'
newish.loc[newish['artist'] == 'Silk City', 'country'] = 'GB/US'
newish.loc[newish['artist'] == 'Vado Más Ki Ás', 'country'] = 'PT'

newish.loc[newish['artist'] == 'ANDREZO', 'country'] = 'PT'
newish.loc[newish['artist'] == 'The Vamps', 'country'] = 'GB'
newish.loc[newish['artist'] == 'LP', 'country'] = 'US'
newish.loc[newish['artist'] == 'DripReport', 'country'] = 'US'
newish.loc[newish['artist'] == 'Miguel', 'country'] = 'US'
newish.loc[newish['artist'] == 'MC\'s Jhowzinho & Kadinho', 'country'] = 'BR'

In [98]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation["artist"].value_counts().head(20)

WC no Beat        14
Leo Santana       14
Brenda Lee        14
NAV               14
Ritt Momney       14
GIMS              13
Rui Orlando       13
Olivia Holt       13
Band Aid          13
Ananda            13
Sam Hunt          13
Andy Williams     13
T-Rex             13
Jennifer Lopez    12
Ir Sais           12
Mia Rose          11
Dj Juninho 22     11
Irina Barros      10
Yuri NR5          10
Conan Osíris      10
Name: artist, dtype: int64

In [99]:
newish.loc[newish['artist'] == 'Conan Osíris', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Yuri NR5', 'country'] = 'PT'
newish.loc[newish['artist'] == 'T-Rex', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Dj Juninho 22', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Joint One', 'country'] = 'PT'

newish.loc[newish['artist'] == 'WC no Beat', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Jennifer Lopez', 'country'] = 'US'
newish.loc[newish['artist'] == 'Leo Santana', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Brenda Lee', 'country'] = 'US'
newish.loc[newish['artist'] == 'NAV', 'country'] = 'CA'
newish.loc[newish['artist'] == 'Ritt Momney', 'country'] = 'US'

newish.loc[newish['artist'] == 'Rui Orlando', 'country'] = 'AN'
newish.loc[newish['artist'] == 'Olivia Holt', 'country'] = 'US'
newish.loc[newish['artist'] == 'Sam Hunt', 'country'] = 'US'

newish.loc[newish['artist'] == 'Andy Williams', 'country'] = 'US'
newish.loc[newish['artist'] == 'Mia Rose', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Ananda', 'country'] = 'BR'

In [100]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation["artist"].value_counts().head(20)

Band Aid           13
GIMS               13
Ir Sais            12
Irina Barros       10
Vice               10
Sage The Gemini    10
Ashe               10
WiNDOH             10
Octavian            9
Guns N' Roses       9
Yung SpaceGhost     9
Clairo              8
Joji                8
Junior Lord         8
Sam Fischer         8
K/DA                8
HMB                 8
S1mba               8
MC Rogerinho        8
Nea                 8
Name: artist, dtype: int64

In [102]:
fix_nation.to_csv("spotify_fixed_nation.csv")